## Setup

In [1]:
import os
import requests
import json
from pathlib import Path

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [2]:
load_eoepca_state()

In [3]:
platform_domain = os.environ.get("INGRESS_HOST")
use_oidc = os.environ.get("OIDC_OAPIP_ENABLED", "true") == "true"
oapip_user = os.environ.get("KEYCLOAK_TEST_USER", "eoepcauser")
oapip_domain = f'https://zoo.{platform_domain}'

# oidc only
oapip_client_id = os.environ.get("OAPIP_CLIENT_ID", "oapip")
oapip_client_secret = os.environ.get("OAPIP_CLIENT_SECRET")
username = os.environ.get("KEYCLOAK_TEST_USER")
password = os.environ.get("KEYCLOAK_TEST_PASSWORD")

## Authentication Setup (Conditional)

In [4]:
if use_oidc:
    # Fetch OIDC Token via Keycloak
    from modules.helpers import get_access_token


    access_token = get_access_token(username, password, oapip_client_id, oapip_client_secret)
    headers = {"Authorization": f"Bearer {access_token}"}
    print("✅ OIDC Authentication setup complete.")
else:
    headers = {}
    print("⚠️ OIDC Authentication not enabled; proceeding without authentication.")


⚠️ OIDC Authentication not enabled; proceeding without authentication.


## Validate API Endpoints

In [5]:
endpoints = [
    ("Swagger UI", f"{oapip_domain}/swagger-ui/oapip/"),
    ("OGC API Processes Landing", f"{oapip_domain}/ogc-api/processes/")
]

for name, url in endpoints:
    response = requests.get(url, headers=headers)
    print(f"{name} ({url}): {response.status_code}")

Swagger UI (https://zoo.test.eoepca.org/swagger-ui/oapip/): 200
OGC API Processes Landing (https://zoo.test.eoepca.org/ogc-api/processes/): 200


## List Available Processes

In [6]:
processes_url = f"{oapip_domain}/{oapip_user}/ogc-api/processes"
response = requests.get(processes_url, headers=headers)

if response.status_code == 200:
    processes = response.json()
    print(f"✅ Available processes:")
    print(json.dumps(processes, indent=2))
else:
    print(f"❌ Failed to retrieve processes. Status code: {response.status_code}")

✅ Available processes:
{
  "processes": [
    {
      "id": "echo",
      "title": "Echo input",
      "description": "Simply echo the value provided as input",
      "mutable": false,
      "version": "2.0.0",
      "metadata": [
        {
          "title": "Demo"
        }
      ],
      "jobControlOptions": [
        "sync-execute",
        "async-execute",
        "dismiss"
      ],
      "outputTransmission": [
        "value",
        "reference"
      ],
      "links": [
        {
          "rel": "self",
          "type": "application/json",
          "title": "Process Description",
          "href": "https://zoo.test.eoepca.org/eoepcauser/ogc-api/processes/echo"
        }
      ]
    }
  ],
  "links": [
    {
      "rel": "self",
      "type": "application/json",
      "href": "https://zoo.test.eoepca.org/eoepcauser/ogc-api/processes"
    }
  ],
  "numberTotal": 1
}


## Deploy a Sample Process (`convert-url`)

In [7]:
deploy_url = processes_url
deploy_payload = {
    "executionUnit": {
        "href": "https://raw.githubusercontent.com/EOEPCA/deployment-guide/refs/heads/main/scripts/processing/oapip/examples/convert-url-app.cwl",
        "type": "application/cwl"
    }
}

response = requests.post(deploy_url, headers={**headers, "Content-Type": "application/json"}, json=deploy_payload)

if response.status_code in [200, 201]:
    print(f"✅ Successfully deployed 'convert-url' process.")
    deployed_process = response.json()
    print(json.dumps(deployed_process, indent=2))
else:
    print(f"❌ Failed to deploy 'convert-url'. Status code: {response.status_code}, Response: {response.text}")

✅ Successfully deployed 'convert-url' process.
{
  "id": "convert-url",
  "title": "convert url app",
  "description": "Convert URL",
  "mutable": true,
  "version": "0.1.2",
  "metadata": [
    {
      "role": "https://schema.org/softwareVersion",
      "value": "0.1.2"
    }
  ],
  "outputTransmission": [
    "value",
    "reference"
  ],
  "jobControlOptions": [
    "async-execute",
    "dismiss"
  ],
  "links": [
    {
      "rel": "http://www.opengis.net/def/rel/ogc/1.0/execute",
      "type": "application/json",
      "title": "Execute End Point",
      "href": "https://zoo.test.eoepca.org/eoepcauser/ogc-api/processes/convert-url/execution"
    }
  ]
}


## Execute the `convert-url` Process

In [8]:
execute_url = f"{processes_url}/convert-url/execution"
execute_payload = {
    "inputs": {
        "fn": "resize",
        "url": "https://eoepca.org/media_portal/images/logo6_med.original.png",
        "size": "50%"
    }
}

response = requests.post(
    execute_url,
    headers={
        **headers,
        "Content-Type": "application/json",
        "Prefer": "respond-async"
    },
    json=execute_payload
)

if response.status_code in [200, 201]:
    job_location = response.headers["Location"]
    job_id = response.json().get("jobID")
    print(f"✅ Execution started successfully. Job ID: {job_id}")
    print(f"Job monitoring URL: {job_location}")
else:
    print(f"❌ Failed to start execution. Status code: {response.status_code}, Response: {response.text}")

✅ Execution started successfully. Job ID: 10924fda-3174-11f0-8181-46173226aeca
Job monitoring URL: https://zoo.test.eoepca.org/eoepcauser/ogc-api/jobs/10924fda-3174-11f0-8181-46173226aeca


## Monitor Execution Status

In [9]:
status_url = f"{oapip_domain}/{oapip_user}/ogc-api/jobs/{job_id}"
status = "running"
import time
print("Monitoring job execution...")
while status.lower() == "running":
    time.sleep(5)
    print("Checking job status...")
    response = requests.get(status_url, headers=headers)
    if response.status_code == 200:
        job_status = response.json()
        status = job_status.get("status")
        print(f"Status: {status}")
        if status.lower() == "running":
            time.sleep(5)  # wait before rechecking
    else:
        print(f"❌ Failed to get job status. Status code: {response.status_code}")
        break

print("Final job status:", status)


Monitoring job execution...
Checking job status...
Status: running
Checking job status...
Status: running
Checking job status...
Status: running
Checking job status...
Status: running
Checking job status...
Status: running
Checking job status...
Status: running
Checking job status...
Status: running
Checking job status...
Status: running
Checking job status...
Status: running
Checking job status...
Status: running
Checking job status...
Status: running
Checking job status...
Status: successful
Final job status: successful


## Retrieve Execution Result

In [10]:
results_url = f"{status_url}/results"
response = requests.get(results_url, headers=headers)

if response.status_code == 200:
    results = response.json()
    print(f"✅ Job results retrieved successfully:")
    print(json.dumps(results, indent=2))
else:
    print(f"❌ Failed to retrieve results. Status code: {response.status_code}, Response: {response.text}")

✅ Job results retrieved successfully:
{
  "type": "Collection",
  "id": "10924fda-3174-11f0-8181-46173226aeca",
  "stac_version": "1.0.0",
  "description": "description",
  "links": [
    {
      "rel": "root",
      "href": "s3://eoepca/processing-results/10924fda-3174-11f0-8181-46173226aeca/catalog.json",
      "type": "application/json"
    },
    {
      "rel": "item",
      "href": "s3://eoepca/processing-results/10924fda-3174-11f0-8181-46173226aeca/processing-results/10924fda-3174-11f0-8181-46173226aeca/logo6_med.original-resize-1747303572.747070150/logo6_med.original-resize-1747303572.747070150.json",
      "type": "application/json"
    },
    {
      "rel": "self",
      "href": "s3://eoepca/processing-results/10924fda-3174-11f0-8181-46173226aeca/processing-results/10924fda-3174-11f0-8181-46173226aeca/collection.json",
      "type": "application/json"
    },
    {
      "rel": "parent",
      "href": "s3://eoepca/processing-results/10924fda-3174-11f0-8181-46173226aeca/catalog.

## Undeploy the `convert-url` Process

In [11]:
undeploy_url = f"{processes_url}/convert-url"
response = requests.delete(undeploy_url, headers=headers)

if response.status_code in [200, 204]:
    print(f"✅ Successfully undeployed 'convert-url' process.")
else:
    print(f"❌ Failed to undeploy 'convert-url'. Status code: {response.status_code}, Response: {response.text}")

✅ Successfully undeployed 'convert-url' process.
